In [ ]:
# librerie
import os
import torch
import random
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import Subset
from data_augmentation import build_balanced_augmented_tensor_dataset
import matplotlib.pyplot as plt
import collections
from torch.utils.data import random_split

In [2]:
# riproducibilità
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
# configurazione
image_size = 224 # dimensione a cui ridimensionare le immagini
noise_std = 0.05 # livello di rumore gaussiano da aggiungere ai dati augmentati
train_ratio, val_ratio = 0.7, 0.15  # percentuali per suddividere in train/val/test

# percorsi utili
base_dir = os.getcwd()  # percorso base
data_dir = os.path.join(base_dir, 'data_histo') # percorso dataset
# percorso per salvare output
output_folder = os.path.join(base_dir, "shared_augmented_data")
os.makedirs(output_folder, exist_ok=True) # crea se non esiste

In [ ]:
# trasformazioni
transform_base = transforms.Compose([
    transforms.Resize((image_size, image_size)),  # ridimensiona le immagini
    transforms.ToTensor(),                        # converte in tensore
])

In [ ]:
# caricamento e split
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform_base)  # carica tutte le immagini con le etichette
total_size = len(full_dataset)                                           # numero totale di immagini
train_size = int(train_ratio * total_size)
val_size = int(val_ratio * total_size)
test_size = total_size - train_size - val_size                           # il resto va al test set

# suddivisione indici casuale per il train set ma riproducibile
train_indices, _, _ = random_split(
    list(range(total_size)), [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

# recupero dei path (e delle classi) delle sole immagini di train
train_samples = [full_dataset.samples[i] for i in train_indices]

In [ ]:
# applicazione data augmentation
print("Generazione dati augmentati...")
train_imgs, train_labels, _ = build_balanced_augmented_tensor_dataset(
    samples=train_samples,
    class_names=full_dataset.classes,
    image_size=image_size,
    mode="balance",             # bilanciamento classi con oversampling
    add_noise=True,             # aggiunta di rumore gaussiano alle immagini
    noise_std=noise_std         # deviazione standard del rumore
)

Generazione dati augmentati...


In [ ]:
# salvataggio dati augmentati (immagini + etichette) per utilizzo futuro
save_path = os.path.join(output_folder, f"augmented_train_data_{noise_std}.pt")
torch.save({
    'images': train_imgs,
    'labels': train_labels
}, save_path)

print(f"Dati augmentati salvati in: {save_path}")

Dati augmentati salvati in: c:\Users\giken\Documents\Noemi\DAML-project\shared_augmented_data\augmented_train_data_0.05.pt
